In [ ]:
#Get gsm8k score

import json
import re


def extract_answer_with_regex(file_path, pattern):
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            answer_text = data['answer']
            match = re.search(pattern, answer_text)
            if match:
                try:
                    value = match.group(1)
                    answers.append(value)
                except ValueError:
                    answers.append(None)
            else:
                answers.append(None)
    return answers

def count_and_report_mismatches(file1, file2):

    pattern1 = r'The answer is:\s*([+-]?\d*[\.,]?\d+)'
    pattern2 = r'####\s*([+-]?\d*[\.,]?\d+)'
    
    answers_file1 = extract_answer_with_regex(file1, pattern1)
    answers_file2 = extract_answer_with_regex(file2, pattern2)
    print(len(answers_file1))
    match_count = 0
    mismatch_indices = []

    for i, (answer1, answer2) in enumerate(zip(answers_file1, answers_file2)):
        if answer1 is not None and answer2 is not None:
            if answer1 == answer2:
                match_count += 1
            else:
                mismatch_indices.append(i + 1)  
        else:
            mismatch_indices.append(i + 1)  
            
    print(f"Number of matches: {match_count}")
    print(f"Line number that does not match: {mismatch_indices}")
    print(match_count/len(answers_file1))
    
    return match_count

file_path = 'sorted_bon_infer_asprm_l_con55_gsm_symbolic_mistral.jsonl'
count_and_report_mismatches(file_path, 'GSM8k_test_idx.jsonl')
import json

total_bon_num = 0
count = 0

with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        if 'bon_num' in data:
            total_bon_num += data['bon_num']
            count += 1
if count > 0:
    average_bon_num = total_bon_num / count
else:
    average_bon_num = 0
print("average_bon_num：",average_bon_num)

In [ ]:
#Get math500 score

import re
import random
import os
import json
import util
import jsonlines

def remove_boxed(s):
    left = "\\boxed{"
    try:
        assert s[:len(left)] == left
        assert s[-1] == "}"
        return s[len(left):-1]
    except:
        return None

def process_results(completion, answer):
    split_ans = completion.split('The answer is: ')
    if len(split_ans) > 1:
        ans = split_ans[-1]
        extract_ans_temp = ans.split('.\n')[0]
        extract_ans_temp = extract_ans_temp.strip()
        if len(extract_ans_temp)>0 and extract_ans_temp[-1] == '.':
            extract_ans = extract_ans_temp[0:-1]
        else:
            extract_ans = extract_ans_temp
        extract_ans = extract_ans.strip()
        if util.is_equiv(extract_ans, answer):
            return True
        else:
            return False
    else:
        return False

if __name__ == "__main__":

    score = 0
    hendrycks_math_answers = []
    solution_list = []
    file_path_math = "sorted_bon_infer_asprm_l_con55_gsm_symbolic_mistral.jsonl"
    with open("math500_test_with_idx.jsonl","r+",encoding="utf8") as file:
        for idx, data in enumerate(jsonlines.Reader(file)):
            gt_answer = data['solution']
            temp_ans = remove_boxed(util.last_boxed_only_string(gt_answer))
            hendrycks_math_answers.append(temp_ans)

    with open(file_path_math, "r+", encoding="utf8") as f:
        for idx, item in enumerate(jsonlines.Reader(f)):
            solution = item['answer']
            solution_list.append(solution)

    results = []

    for idx, (completion, prompt_answer) in enumerate(zip(solution_list, hendrycks_math_answers)):
        res = process_results(completion, prompt_answer)
        results.append(res)


    final_score = sum(results)/len(results)

    print(final_score)



In [ ]:
#Get gsm8k_symbolic score

import json
import re

def extract_answer_with_regex(file_path, pattern):
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            if 'gt_answer' in data:
                answer_text = data['gt_answer']
            elif 'answer' in data:
                answer_text = data['answer']
            match = re.search(pattern, answer_text)
            if match:
                try:
                    value = match.group(1)
                    answers.append(value)
                except ValueError:
                    answers.append(None)
            else:
                answers.append(None)
    return answers

def count_and_report_mismatches(file1, file2):

    pattern1 = r'The answer is:\s*([+-]?\d*[\.,]?\d+)'

    pattern2 = r'####\s*([+-]?\d*[\.,]?\d+)'

    
    answers_file1 = extract_answer_with_regex(file1, pattern1)
    answers_file2 = extract_answer_with_regex(file2, pattern2)
    print(len(answers_file1))
    match_count = 0
    mismatch_indices = []

    for i, (answer1, answer2) in enumerate(zip(answers_file1, answers_file2)):
        if answer1 is not None and answer2 is not None:
            if answer1 == answer2:
                match_count += 1
            else:
                mismatch_indices.append(i + 1)  
        else:
            mismatch_indices.append(i + 1)  

    print(f"Number of matches: {match_count}")
    print(f"Line number that does not match: {mismatch_indices}")
    print(match_count/len(answers_file1))
    return match_count

file_path = 'sorted_bon_infer_asprm_l_con55_gsm_symbolic_mistral.jsonl'
count_and_report_mismatches(file_path, 'GSM_symbolic_test_idx.jsonl')

total_bon_num = 0
count = 0

with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        if 'bon_num' in data:
            total_bon_num += data['bon_num']
            count += 1

if count > 0:
    average_bon_num = total_bon_num / count
else:
    average_bon_num = 0
print("average_bon_num：",average_bon_num)